In [ ]:
!pip install transformers bitsandbytes accelerate

In [ ]:
from torch import cuda, bfloat16
import transformers

model_id = "NumbersStation/nsql-llama-2-7B"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'


## 4bit quantiazation
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)



model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto'
)

In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto'
)

In [8]:
### Basic Examples
text = """CREATE TABLE stadium (
    stadium_id number,
    location text,
    name text,
    capacity number,
)

-- Using valid SQLite, answer the following questions for the tables provided above.

-- how many unique location in total?

SELECT"""


input_ids = tokenizer(text, return_tensors="pt").input_ids
input_ids = input_ids.to('cuda')
generated_ids = model.generate(input_ids, max_length=500,do_sample=True)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

CREATE TABLE stadium (
    stadium_id number,
    location text,
    name text,
    capacity number,
)

-- Using valid SQLite, answer the following questions for the tables provided above.

-- how many unique location in total?

SELECT COUNT(DISTINCT location) FROM stadium


In [15]:
### Basic Examples
text = """CREATE TABLE stadium (
    stadium_id number,
    location text,
    name text,
    capacity number,
)

-- Using valid SQLite, answer the following questions for the tables provided above.

-- find the name = Alpha from this table?

SELECT"""


input_ids = tokenizer(text, return_tensors="pt").input_ids
input_ids = input_ids.to('cuda')
generated_ids = model.generate(input_ids, max_length=500,do_sample=True)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

CREATE TABLE stadium (
    stadium_id number,
    location text,
    name text,
    capacity number,
)

-- Using valid SQLite, answer the following questions for the tables provided above.

-- find the name = Alpha from this table?

SELECT name FROM stadium WHERE name = 'Alpha'


In [9]:
### Check Grouping and Aggregate performance
text = """CREATE TABLE stadium (
    stadium_id number,
    location text,
    name text,
    capacity number,
)

-- Using valid SQLite, answer the following questions for the tables provided above.

-- how many maximum capacity for each group?

SELECT"""


input_ids = tokenizer(text, return_tensors="pt").input_ids
input_ids = input_ids.to('cuda')
generated_ids = model.generate(input_ids, max_length=500,do_sample=True)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

CREATE TABLE stadium (
    stadium_id number,
    location text,
    name text,
    capacity number,
)

-- Using valid SQLite, answer the following questions for the tables provided above.

-- how many maximum capacity for each group?

SELECT MAX(capacity), name FROM stadium GROUP BY name


In [10]:
### Check Grouping and Aggregate performance
text = """CREATE TABLE stadium (

    country text,
    city text,
    capacity number,
)

-- Using valid SQLite, answer the following questions for the tables provided above.

-- how many maximum and minimum capacity for each group of city and country?

SELECT"""


input_ids = tokenizer(text, return_tensors="pt").input_ids
input_ids = input_ids.to('cuda')
generated_ids = model.generate(input_ids, max_length=500,do_sample=True)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

CREATE TABLE stadium (

    country text,
    city text,
    capacity number,
)

-- Using valid SQLite, answer the following questions for the tables provided above.

-- how many maximum and minimum capacity for each group of city and country?

SELECT MAX(capacity), MIN(capacity), country, city FROM stadium GROUP BY country, city


In [12]:
### Check Join Operation
text = """
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    department_id INTEGER,
    FOREIGN KEY (department_id) REFERENCES departments(id)
);

CREATE TABLE departments (
    id INTEGER PRIMARY KEY,
    department_name TEXT NOT NULL
);


-- Using valid SQLite, answer the following questions for the tables provided above.

-- list all employees along with their department names using join?

SELECT"""


input_ids = tokenizer(text, return_tensors="pt").input_ids
input_ids = input_ids.to('cuda')
generated_ids = model.generate(input_ids, max_length=500,do_sample=True)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))


CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    department_id INTEGER,
    FOREIGN KEY (department_id) REFERENCES departments(id)
);

CREATE TABLE departments (
    id INTEGER PRIMARY KEY,
    department_name TEXT NOT NULL
);


-- Using valid SQLite, answer the following questions for the tables provided above.

-- list all employees along with their department names using join?

SELECT employees.name, departments.department_name FROM employees JOIN departments ON employees.department_id = departments.id
